In [1]:
import pandas as pd

In [2]:
# load data with specified bad values as NaN
bad_values = ['unknown', 'n/a', '--', 'NA', 'NaN', '', '999', 'Invalid', 'Unknown', 'N/A']
df = pd.read_csv('messy_customer_data.csv', na_values=bad_values)

In [3]:
# drop duplicates and create order_id column to fix the dataset
df.drop_duplicates(inplace=True)
df['order_id'] = range(1, len(df) + 1)

In [4]:
# Replace NaN with mean purchase amount
df["purchase_amount"].fillna(df["purchase_amount"].mean(), inplace=True)

C:\Users\joao2\AppData\Local\Temp\ipykernel_27492\3237769482.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["purchase_amount"].fillna(df["purchase_amount"].mean(), inplace=True)


In [5]:
# Standardize names
df["name"] = df["name"].str.lower().str.capitalize()
df.sample(5)

,customer_id,name,age,signup_date,city,purchase_amount,vip,comments,order_id
23,CUST0049,Alice,40.0,01/02/2023,New York,300.000000,Y,bad,24
4,CUST0072,Alice,35.0,01/02/2023,Los Angeles,291.780822,No,NaN,5
49,CUST0081,Bob,NaN,NaN,New York,100.000000,No,NaN,50
21,CUST0022,Alice,27.0,01/02/2023,new york,200.000000,N,bad,22
94,CUST0048,Charlie,50.0,NaN,LA,200.000000,No,bad,95


In [6]:
# Standardize city names
city_map = {
    "new york": "New York",
    "nyc": "New York",
    "los angeles": "Los Angeles",
    "la": "Los Angeles"
}

df.loc[:, 'city'] = df["city"].str.lower().map(city_map).fillna(df["city"]) 

In [7]:
# Convert age type to integer
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['age'] = df['age'].astype('Int64')
# df['age'] = df['age'].fillna(df['age'].median())


In [8]:
# Convert purchase_amount to numeric and round to 2 decimal places
df['purchase_amount'] = pd.to_numeric(df['purchase_amount'], errors='coerce')
df['purchase_amount'] = df['purchase_amount'].round(2)

In [9]:
# drop orders with purchase_amount less than 0
# You can keep it if needed
df = df[df['purchase_amount'] >= 0 ]


In [10]:
# Standardize VIP status
vip_std = {
    'n': 'No',
    'no': 'No',
    'y': 'Yes',
    'yes': 'Yes'
}

df.loc[:, 'vip'] = df['vip'].str.lower().map(vip_std).fillna('No')


In [11]:
# Convert signup_date to datetime
df['signup_date'] = pd.to_datetime(df['signup_date'], errors='coerce', format='mixed')


In [12]:
# Reorder columns
current_cols = df.columns.tolist()
columns = ['order_id', 'customer_id', 'name', 'age', 'signup_date', 'city', 'purchase_amount', 'vip', 'comments']
df = df[columns]

In [13]:
df.sample(5)

,order_id,customer_id,name,age,signup_date,city,purchase_amount,vip,comments
68,69,CUST0053,Alice,25,NaT,New York,291.78,No,NaN
114,115,CUST0052,Alice,30,NaT,New York,291.78,Yes,NaN
113,114,CUST0096,Alice,30,2023-01-01,New York,291.78,Yes,ok
100,101,CUST0101,Alice,<NA>,NaT,Los Angeles,291.78,No,ok
44,45,CUST0084,Bob,40,NaT,New York,100.00,No,NaN


In [14]:
# Create new column signup_year
df['signup_year']= df['signup_date'].dt.year
df['signup_year'] = df['signup_year'].astype('Int64', errors='ignore')

In [15]:
df.reset_index(drop=True, inplace=True)

In [16]:
df.to_csv('cleaned_customer_data.csv', index=False)